In [2]:
import tkinter as tk
import serial
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

#UART-Verbindung erstellen (Port 7)
ser = serial.Serial('COM4', 9600)

time_values = []
position_values = []
counter = 0

def toggle_measurement(*args):
    global counter
    global messung_aktiv
    """Sendet 'M' beim Aktivieren und 'MN' beim Deaktivieren"""
    if abstand_var.get():
        ser.write("M\n".encode()) # Messung starten
        messung_aktiv = 1
    else:
        ser.write("MN\n".encode()) 
        messung_aktiv = 0
        ser.reset_input_buffer()
def send_text():
    text = text_entry.get().strip()
    if text:  # Falls Text vorhanden ist
        ser.write(f"Beschriftung:{text}\n".encode())  # Sende den eingegebenen Text
    else:
        print("Kein Text gesendet")
        
def toggle_ref(*args):
    if ref_var.get():
        ser.write("R\n".encode())  # Referenzfahrt starten
    else:
        print("keine Referenzfahrt")

def start_action():
    """Sendet Start-Signal an Arduino"""
    ser.write("START\n".encode())
    
def stop_action():
    """Sendet Not-Aus-Signal an Arduino"""
    ser.write("STOP\n".encode())
    
def receive_data():
    """Empfängt Daten vom Arduino und unterscheidet zwischen Text und Messwerten"""
    global counter  # Damit die Funktion den globalen Zähler verändert

    if ser.in_waiting > 0:
        received_message = ser.readline().decode("latin-1").strip()
        try:
            # Prüft, ob die Nachricht "mm" enthält
            if "mm" in received_message and messung_aktiv:
                distance = int(re.search(r'\d+', received_message).group())  # Konvertiert in eine Ganzzahl
                # Werte speichern
                time_values.append(counter)  # Zeitwert hinzufügen
                position_values.append(distance)  # Messwert speichern
                counter += 1  # Zähler erhöhen
                update_plot()
            else:
                # Falls keine gültige Messwertnachricht, nur ausgeben
                print(f"{received_message}")

        except ValueError:
            # Falls die Konvertierung fehlschlägt, ist es kein gültiger Messwert
            print(f"Fehlerhafte Nachricht: {received_message}")
    
    root.after(100, receive_data)  # Wiederholung alle 100 ms

def update_plot():
    """Aktualisiert das Diagramm in der GUI"""
    ax.clear()
    ax.plot(time_values, position_values, marker="x", linestyle="-", color="red")
    ax.set_title("Abstandsmessung über Zeit")
    ax.set_xlabel("Zeit")
    ax.set_ylabel("Position")
    canvas.draw()

# Hauptfenster
root = tk.Tk()
root.title("Arduino Steuerung")

# Checkbox für Referenzfahrt
ref_var = tk.BooleanVar()
ref_checkbox = tk.Checkbutton(root, text="Referenzfahrt", variable=ref_var)
ref_checkbox.pack()
ref_var.trace_add("write", toggle_ref)

# Checkbox für Abstand messen
abstand_var = tk.BooleanVar()
abstand_checkbox = tk.Checkbutton(root, text="Abstand messen", variable=abstand_var)
abstand_checkbox.pack()
abstand_var.trace_add("write", toggle_measurement)

# Matplotlib Diagramm
fig, ax = plt.subplots()
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().pack()

# Eingabefeld für den Text
text_entry = tk.Entry(root)
text_entry.pack()

# Button zum Senden
send_button = tk.Button(root, text="Sende Text", command=send_text)
send_button.pack()

# Button zum Starten
start_button = tk.Button(root, text="Start", command=start_action)
start_button.pack()

# Button für Not-Aus
stop_button = tk.Button(root, text="Not-Aus", command=stop_action, bg="red", fg="white")
stop_button.pack()

# Startet die Datenempfangsfunktion
root.after(100, receive_data)

root.mainloop()


ModuleNotFoundError: No module named 'serial'